### Chemical Space Analysis - PCA, tSNE, Murcko frameworks

Developed by Joyce Borba 

### Import the Necessary Libraries




In [ ]:
import os
import time
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmilesFromSmiles
from rdkit import Chem, DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.PandasTools import ChangeMoleculeRendering

import mols2grid
from tqdm.auto import tqdm
from ipywidgets import widgets
from typing import List

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
#import hdbscan

import matplotlib.pyplot as plt
import seaborn as sns
IPythonConsole.ipython_useSVG=True 
from IPython.display import SVG

#Bokeh library for plotting
import json
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.transform import factor_cmap
from bokeh.transform import transform
from bokeh.transform import LinearColorMapper
from bokeh.models import ColorBar
from bokeh.palettes import PiYG
from bokeh.plotting import figure, output_file, save
output_notebook()



Enable Pandas **progress_apply**

In [ ]:
tqdm.pandas()

### Setup

Read the SMILES data

In [ ]:
fname = r"C:\Users\franc\OneDrive\Documentos\LabMol\IC-Citotoxicidade\datasets\AID_1345082 3T3\BALANCED\curated_reduced_com_smiles.csv"

In [ ]:
#df = PandasTools.LoadSDF(fname, smilesName='SMILES', includeFingerprints=False)
#df.info()

In [ ]:
df = pd.read_csv(fname)
df.info()

In [ ]:
df=df.rename(columns={'Outcome':'ACTIVITY'})
df.info()

In [ ]:
df1=df[['SMILES','ACTIVITY']]
df1.info()

In [ ]:
a =pd.DataFrame(df)
a

In [ ]:
df1.isnull().any()

In [ ]:
df1=df1.dropna(subset=['ACTIVITY'])

In [ ]:
df1.info()

In [ ]:
df1['mol'] = df.SMILES.progress_apply(Chem.MolFromSmiles)

In [ ]:
df1.isnull().any()

In [ ]:
df1.info()

In [ ]:
df2=df1.dropna(subset=['mol'])
df2.info()

In [ ]:
df2['framework'] = df2.SMILES.progress_apply(MurckoScaffoldSmilesFromSmiles)

### Create a DataFrame to Organize the Scaffolds



As the first part of our analysis, we'd like to look at how frequently each scaffold occurs.  We used the groupby function, agregating via framework - "size" counts the frequency and we also calculate the mean activity for further analysis.

In [ ]:
scaffold_df = df2.groupby('framework').agg({'framework':'size', 'ACTIVITY':'mean'})
scaffold_df

In [ ]:
#reseting the index and sorting values for the most present scaffolds
scaffold_df=scaffold_df.rename(columns={'framework':'n_mol'})
scaffold_df=scaffold_df.reset_index()
scaffold_df['scaffold_idx'] = scaffold_df.index
scaffold_df=scaffold_df.rename(columns={'framework':'scaffold'})
scaffold_df=scaffold_df.sort_values(by=['n_mol'], ascending=False)
scaffold_df.head()

In [ ]:
scaffold_df.to_csv('scaffolds_ames_grouped.csv')

In [ ]:
df2.to_csv('ames_cpds_scaffods.csv')

In [ ]:
#tirei os dados sem scaffold e o benzeno pro plot ficar mais bonito e informativo:
scaffold_df = scaffold_df[scaffold_df.scaffold != '']
scaffold_df = scaffold_df[scaffold_df.scaffold != 'c1ccccc1']
scaffold_df.head()

### Calculate descriptors (morgan)

In [ ]:
scaffold_df['mol'] = scaffold_df.scaffold.progress_apply(Chem.MolFromSmiles)

In [ ]:
scaffold_df=scaffold_df.loc[scaffold_df.mol.notnull()]

In [ ]:
scaffold_df.info()

In [ ]:
scaffold_df

In [ ]:
#calculate ECF4 descriptor
ECFP4_fps = np.array([AllChem.GetMorganFingerprintAsBitVect(x,2) for x in scaffold_df['mol']])


### Calculate LDA


In [ ]:
x_train_lda = ECFP4_fps
y_train_lda = scaffold_df["n_mol"]

In [ ]:
x_train_lda = StandardScaler().fit_transform(x_train_lda)

In [ ]:
%%time
lda = LDA(n_components=2)
lda.fit(x_train_lda, y_train_lda)
x_lda = lda.transform(x_train_lda)
lda_df = pd.DataFrame(x_lda, columns=['X_LDA','Y_LDA'])

lda_df.info()

### Calculate PCA

In [ ]:
%%time
pca = PCA(n_components=2)
X_pca = pca.fit_transform(ECFP4_fps)
pca_df = pd.DataFrame(X_pca, columns= ['X_PCA','Y_PCA'])
pca_df.info()

### Calculate TSNE

In [ ]:
%%time
tsne = TSNE(random_state=0).fit_transform(ECFP4_fps)
#tsne = TSNE(random_state=0).fit_transform(ECFP4_fps)
tsne_df = pd.DataFrame(tsne, columns= ['X_TSNE', 'Y_TSNE'])

tsne_df.info()

### Calculate UMAP

### Prepare molecules to print

In [ ]:
def _prepareMol(mol,kekulize):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    return mc

def moltosvg(mol,molSize=(450,200),kekulize=True,drawer=None,**kwargs):
    mc = _prepareMol(mol,kekulize)
    if drawer is None:
        drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
    drawer.DrawMolecule(mc,**kwargs)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    return SVG(svg.replace('svg:',''))

In [ ]:
svgs = [moltosvg(m).data for m in scaffold_df.mol]

In [ ]:
scaffold_df.info()

In [ ]:
scaffold_df=scaffold_df.reset_index()
scaffold_df.head()

In [ ]:
df_merge = pd.concat([scaffold_df, pca_df, tsne_df, lda_df], axis=1)
df_merge.head()

### Plot interactive maps

In [ ]:
color_mapper=LinearColorMapper(palette=PiYG[9],
                               low=scaffold_df.ACTIVITY.max(), high=scaffold_df.ACTIVITY.min())

In [ ]:
scaffold_df['s']= [m for m in scaffold_df.n_mol]

In [ ]:
scaffold_df

In [ ]:
def plot_int_map(metodology):
    source = ColumnDataSource(data=dict(x=metodology[:,0], y=metodology[:,1], freq = scaffold_df.n_mol, desc= scaffold_df.ACTIVITY,
                                    svgs=svgs, s=scaffold_df.s*6, c=scaffold_df.ACTIVITY))
    ChangeMoleculeRendering(renderer='PNG')



    hover = HoverTool(tooltips="""
        <div>
            <div>@svgs{safe}
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">Frequency @freq</span>
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">Pred_True_Positive @desc</span>
            </div>
        </div>
        </body>
        """)

    interactive_map = figure(width = 1000, height=1000, tools=['reset,box_zoom,wheel_zoom,zoom_in,zoom_out,pan',hover],
           title="Chemical Space of external set (true and false positives)")



    interactive_map.circle('x', 'y', 
                           source=source,
                           size='s',
                           color=transform('c', color_mapper),
                           fill_alpha=0.5);

    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title ='Activity')
    interactive_map.add_layout(color_bar,'left')
    show(interactive_map)
    return interactive_map

In [ ]:
pca_map=plot_int_map(X_pca)

In [ ]:
lda_map=plot_int_map(x_lda)

In [ ]:
tsne_map=plot_int_map(tsne)

In [ ]:
output_file("./tsne_ames_FP.html")
save(tsne_map)

output_file("./pca_ames_FP.html")
save(pca_map)

output_file("./lda_ames_FP.html")
save(lda_map)

In [ ]:
scaffold_df.to_csv('./scaffolds_ames_CS_FP.csv')